<a href="https://colab.research.google.com/github/SawairaK/Sentiment_Analysis_using_Transformer/blob/main/fine_tuning_using_distilbert_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.30
!pip install datasets

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments

In [ ]:
#Load data after Pre_processing
df = pd.read_csv('cleaned.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df

,review_body,product_category,count,sentiment
0,If this product was in petite i would get the...,Blouses,60,2
1,Love this dress! it's sooo pretty. i happene...,Dresses,62,2
2,I had such high hopes for this dress and reall...,Dresses,98,1
3,I love love love this jumpsuit. it's fun fl...,Pants,22,2
4,This shirt is very flattering to all due to th...,Blouses,36,2
...,...,...,...,...
22615,I was very happy to snag this dress at such a ...,Dresses,28,2
22616,It reminds me of maternity clothes. soft stre...,Knits,38,1
22617,This fit well but the top was very see throug...,Dresses,42,1
22618,I bought this dress for a wedding i have this ...,Dresses,86,1


In [ ]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification

##Load model
model_name = "distilbert-base-uncased"  # Replace with a valid model name from Hugging Face's model hub
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_

In [ ]:
df

,review_body,product_category,count,sentiment
0,If this product was in petite i would get the...,Blouses,60,2
1,Love this dress! it's sooo pretty. i happene...,Dresses,62,2
2,I had such high hopes for this dress and reall...,Dresses,98,1
3,I love love love this jumpsuit. it's fun fl...,Pants,22,2
4,This shirt is very flattering to all due to th...,Blouses,36,2
...,...,...,...,...
22615,I was very happy to snag this dress at such a ...,Dresses,28,2
22616,It reminds me of maternity clothes. soft stre...,Knits,38,1
22617,This fit well but the top was very see throug...,Dresses,42,1
22618,I bought this dress for a wedding i have this ...,Dresses,86,1


In [ ]:
texts = df['review_body'].to_list()
labels = df['sentiment'].to_list()


## Train test split

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.1, random_state=0, shuffle=True)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size=0.1, random_state=0, shuffle=True)



In [ ]:
test_data = pd.DataFrame(test_texts)
test_data.to_csv('./test_dataset.csv')

In [ ]:
test_label = pd.DataFrame(test_labels)
test_label.to_csv('./test_datalabels.csv')

In [ ]:
len(train_texts), len(val_texts),len(test_texts)

(18322, 2262, 2036)

In [ ]:
class ClothDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)


In [ ]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Set the maximum sequence length
max_length = 128

train_encodings = tokenizer(train_texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
test_encodings = tokenizer(test_texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')

train_dataset = ClothDataset(train_encodings, train_labels)
val_dataset = ClothDataset(val_encodings, val_labels)
test_dataset = ClothDataset(test_encodings, test_labels)

In [ ]:
pip install accelerate

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mo

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    print("GPU is available and PyTorch is using it.")
else:
    print("No GPU available. PyTorch is using CPU.")

model = model.to(device)



GPU is available and PyTorch is using it.


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-20-7bff2a381b02>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,1.001000
20,0.987600
30,0.913400
40,0.887700
50,0.865100
60,0.778400
70,0.753900
80,0.702800
90,0.772400
100,0.674700


<ipython-input-20-7bff2a381b02>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-20-7bff2a381b02>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-20-7bff2a381b02>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-20-7bff2a381b02>:7: UserWarning: To copy construct from a ten

TrainOutput(global_step=11460, training_loss=0.15274402055744915, metrics={'train_runtime': 2256.8073, 'train_samples_per_second': 81.185, 'train_steps_per_second': 5.078, 'total_flos': 6067777403704320.0, 'train_loss': 0.15274402055744915, 'epoch': 10.0})

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from datasets import load_dataset
from evaluate import load # Import load from evaluate instead of datasets

def compute_metrics(eval_pred):
    # load the metrics to use
    load_accuracy = load("accuracy") # Use load from evaluate
    load_f1 = load("f1") # Use load from evaluate

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculate the mertic using the predicted and true value
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)
    f1 = load_f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": accuracy, "f1score": f1}
    # return {"accuracy": accuracy}

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,

)

In [ ]:
# Launch the final evaluation
trainer.evaluate()

<ipython-input-20-7bff2a381b02>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}


Trainer is attempting to log a value of "{'accuracy': 0.8521611001964636}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8568803547264661}" of type <class 'dict'> for key "eval/f1score" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 1.1561967134475708,
 'eval_accuracy': {'accuracy': 0.8521611001964636},
 'eval_f1score': {'f1': 0.8568803547264661},
 'eval_runtime': 11.0954,
 'eval_samples_per_second': 183.5,
 'eval_steps_per_second': 2.884}

In [ ]:
import os
# Load the fine-tuned model and tokenizer
model.save_pretrained("/content/results/models")
tokenizer.save_pretrained("/content/results/models")
os.chdir('/content/results/models')
